# Source: Sports Reference - NFL Data

In [1]:
#hide
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql

## Set Notebook Settings

In [2]:
years = cc.get_defYears()
headers= cc.get_header()

## Get, Process the NFL draft data from SR
#### Source: https://www.pro-football-reference.com/years/2017/draft.htm
> This page contains metadata of each draft pick, both in terms of draft position but also current pro stats.

In [ ]:
with open("..//scrapedData//nfldraft.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.handle_nflData(years, headers)))

## Summarize the Dataset

> We don't need repetitive fields across the various datasets (I don't need height coming back from 3 sources - just don't care).  This steps strips to only what I care about for the master print out.

In [ ]:
outputDirectory = '..//summarizedData//'
dataset = 'nflData'

with open(outputDirectory + dataset + ".json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.summarize_nflDraft()))

## Save to DB

In [3]:
fx.toDB_NFLDraft()

'DB Write is done'